In [26]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras.utils import to_categorical
import numpy as np
import os

In [27]:
n = 3
version = 1
depth = n * 6 +2
model_type = 'ResNet%dv%d' % (depth, version)

In [28]:
# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [29]:
# Input image dimensions.
input_shape = x_train.shape[1:]
input_shape

(32, 32, 3)

In [30]:
# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [31]:
# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

In [32]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [33]:
# Convert class vectors to binary class matrices.

num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [34]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [35]:
def resnet(input_shape, depth, num_classes=10):
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Compile the Model

In [36]:
model = resnet(input_shape=input_shape, depth=depth)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=1e-3),
              metrics=['accuracy'])

In [37]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_43 (Conv2D)              (None, 32, 32, 16)   448         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_39 (BatchNo (None, 32, 32, 16)   64          conv2d_43[0][0]                  
__________________________________________________________________________________________________
activation_39 (Activation)      (None, 32, 32, 16)   0           batch_normalization_39[0][0]     
____________________________________________________________________________________________

In [38]:
# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

In [39]:
# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True,
                             mode='auto'
                            )

# Train the Model

In [41]:
model.fit(x_train, y_train,
              batch_size=32,
              epochs=10,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=[checkpoint])

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 667s 13ms/step - loss: 1.4580 - accuracy: 0.5309 - val_loss: 1.4002 - val_accuracy: 0.5644
Epoch 2/10


C:\Users\raiya\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


50000/50000 [==============================] - 656s 13ms/step - loss: 1.0553 - accuracy: 0.6831 - val_loss: 1.1681 - val_accuracy: 0.6408
Epoch 3/10
50000/50000 [==============================] - 655s 13ms/step - loss: 0.8957 - accuracy: 0.7441 - val_loss: 1.1507 - val_accuracy: 0.6661
Epoch 4/10
50000/50000 [==============================] - 652s 13ms/step - loss: 0.7952 - accuracy: 0.7800 - val_loss: 0.9888 - val_accuracy: 0.7303
Epoch 5/10
50000/50000 [==============================] - 653s 13ms/step - loss: 0.7333 - accuracy: 0.8068 - val_loss: 1.1187 - val_accuracy: 0.6985
Epoch 6/10
50000/50000 [==============================] - 650s 13ms/step - loss: 0.6758 - accuracy: 0.8289 - val_loss: 0.8890 - val_accuracy: 0.7578
Epoch 7/10
50000/50000 [==============================] - 649s 13ms/step - loss: 0.6361 - accuracy: 0.8430 - val_loss: 1.1041 - val_accuracy: 0.7184
Epoch 8/10
50000/50000 [==============================] - 650s 13ms/step - loss: 0.6051 - accuracy: 0.8569 - val_loss

# Save the Model

In [42]:
fname = "weights-ResNetImageClassification.hdf5"
model.save_weights(fname,overwrite=True)

In [43]:
fname = "weights-ResNetImageClassification.hdf5"
model.load_weights(fname)

# Score trained Model

In [44]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 40s 4ms/step
Test loss: 0.8247979865074158
Test accuracy: 0.7961000204086304
